In [48]:
import pickle
import os
import pandas as pd
import numpy as np
import requests
import plotly.express as px
import plotly.io as pio
import math
from geojson_rewind import rewind
pio.renderers.default = 'iframe'

In [1]:
def build_path(subfolder = 'merge'):
    folderpath = os.path.join(os.getcwd(), os.pardir, 
                              'project', 'data', subfolder)
    folderpath = os.path.abspath(folderpath)
    if not os.path.exists(folderpath): 
        os.makedirs(folderpath)
    return folderpath

def load_data():
    data = {}   
    folderpath = build_path()
    for filename in os.listdir(folderpath):
        if filename.endswith('.json'):
            filepath = os.path.join(build_path('merge'), filename)
            data[filename.split('.')[0]] = pd.read_json(filepath)
    return data

def save_data(figure, type_of_map, intraregion, subfolder = 'app'):
    filepath = os.path.join(build_path(subfolder), f'{type_of_map}-{intraregion}.pkl')
    
    with open(filepath, 'wb') as pickle_file:
        pickle.dump(figure, pickle_file)      

In [19]:
variaveis_uf_ano = pd.read_csv('variaveis_uf_ano.csv', sep=';')
variaveis_uf_ano

,uf,ano,gastos,gastos_percapita,mortalidade,valor_emendas
0,AC,2014,3.603932e+07,337.690253,0.920833,1.261524e+08
1,AC,2015,3.726202e+07,305.003132,1.236364,2.648756e+07
2,AC,2016,4.048715e+07,301.639801,1.342000,4.542464e+08
3,AC,2017,3.724553e+07,274.220778,1.467500,2.436925e+08
4,AC,2018,3.443257e+07,263.785812,1.718000,1.976735e+08
...,...,...,...,...,...,...
238,TO,2018,6.298209e+07,894.183551,1.939130,1.021493e+09
239,TO,2019,8.231413e+07,1001.786091,2.390476,5.708286e+09
240,TO,2020,9.288584e+07,1042.532604,2.676364,1.037564e+10
241,TO,2021,1.192244e+08,1455.791855,3.041538,4.138913e+09


In [25]:
# Criando UF e região
CondUF =  [(variaveis_uf_ano['uf'] == 'RO'),
         (variaveis_uf_ano["uf"] == 'AC'),
         (variaveis_uf_ano["uf"] == 'AM'),
         (variaveis_uf_ano["uf"] == 'RR'),
         (variaveis_uf_ano["uf"] == 'PA'),
         (variaveis_uf_ano["uf"] == 'AP'),
         (variaveis_uf_ano["uf"] == 'TO'),
         (variaveis_uf_ano["uf"] == 'MA'),
         (variaveis_uf_ano["uf"] == 'PI'),
         (variaveis_uf_ano["uf"] == 'CE'),
         (variaveis_uf_ano["uf"] == 'RN'),
         (variaveis_uf_ano["uf"] == 'PB'),
         (variaveis_uf_ano["uf"] == 'PE'),
         (variaveis_uf_ano["uf"] == 'AL'),
         (variaveis_uf_ano["uf"] == 'SE'),
         (variaveis_uf_ano["uf"] == 'BA'),
         (variaveis_uf_ano["uf"] == 'MG'),
         (variaveis_uf_ano["uf"] == 'ES'),
         (variaveis_uf_ano["uf"] == 'RJ'),
         (variaveis_uf_ano["uf"] == 'SP'),
         (variaveis_uf_ano["uf"] == 'PR'),
         (variaveis_uf_ano["uf"] == 'SC'),
         (variaveis_uf_ano["uf"] == 'RS'),
         (variaveis_uf_ano["uf"] == 'MS'),
         (variaveis_uf_ano["uf"] == 'MT'),
         (variaveis_uf_ano["uf"] == 'GO'),
         (variaveis_uf_ano["uf"] == 'DF')
         ]
          
UF_nomes = ['11','12','13','14','15','16','17','21','22','23','24','25','26','27','28','29','31','32','33','35','41','42','43','50','51','52','53']
variaveis_uf_ano["cod_UF"] = np.select(CondUF, UF_nomes, default="Error")

variaveis_uf_ano

,uf,ano,gastos,gastos_percapita,mortalidade,valor_emendas,cod_UF
0,AC,2014,3.603932e+07,337.690253,0.920833,1.261524e+08,12
1,AC,2015,3.726202e+07,305.003132,1.236364,2.648756e+07,12
2,AC,2016,4.048715e+07,301.639801,1.342000,4.542464e+08,12
3,AC,2017,3.724553e+07,274.220778,1.467500,2.436925e+08,12
4,AC,2018,3.443257e+07,263.785812,1.718000,1.976735e+08,12
...,...,...,...,...,...,...,...
238,TO,2018,6.298209e+07,894.183551,1.939130,1.021493e+09,17
239,TO,2019,8.231413e+07,1001.786091,2.390476,5.708286e+09,17
240,TO,2020,9.288584e+07,1042.532604,2.676364,1.037564e+10,17
241,TO,2021,1.192244e+08,1455.791855,3.041538,4.138913e+09,17


In [74]:
# Puxar a malha geográfica do Brasil a nível de município
geojson = requests.get(f'https://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=maxima&intrarregiao=UF').json()

# A malha geográfica do IBGE tem 7 dígitos (remover o dígito verificador e atualizar a malha)
for feature in geojson['features']:
    feature['properties']['codarea'] = feature['properties']['codarea']
geojson = rewind(geojson, rfc7946=False)


# Construir o mapa choroplético com uma barra de cores fixa e sem variação
fig = px.choropleth(variaveis_uf_ano,
                    geojson=geojson,
                    scope='south america',
                    color='gastos_percapita',
                    color_continuous_scale="agsunset_r",  # Escolha um mapa de cores adequado aqui
                    color_continuous_midpoint=5_00,  # Defina um valor médio para o mapa de cores
                    range_color=[0, 25_000],  # Defina manualmente o intervalo de cores
                    locations='cod_UF',
                    featureidkey='properties.codarea',
                    hover_name='uf',
                    animation_frame='ano')

fig.update_geos(fitbounds="locations", visible=False)
fig.update_traces(marker_line_width=0)

fig.update_layout(height=800, width=1000, autosize=False, 
                  updatemenus=[{"buttons": [{"args": [None, {"frame": {"duration": 1000, "redraw": True}, "fromcurrent": True, "mode": "immediate", "transition": {"duration": 500, "easing": "linear"}}], "label": "Play", "method": "animate"}, 
                                          {"args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}], "label": "Stop", "method": "animate"}], 
                               "direction": "left", "pad": {"r": 10, "t": 87}, "showactive": False, "type": "buttons", "x": 0.1, "xanchor": "right", "y": 0, "yanchor": "top"}])

fig.show()

In [38]:
max=variaveis_uf_ano['gastos'].max()
print(max)

5067663861.0


In [39]:
min=variaveis_uf_ano['gastos'].min()
print(min)

19757952.27


In [3]:
import pickle

def save_data(fig):
    arquivo = 'C:/Users/Rosana/Documents/Rosana/Bootcamp_saude/fig.pkl'
    with open(arquivo, 'wb') as pickle_file:
        pickle.dump(fig, pickle_file)

In [77]:
# Puxar a malha geográfica do Brasil a nível de município
geojson = requests.get(f'https://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=maxima&intrarregiao=UF').json()

# A malha geográfica do IBGE tem 7 dígitos (remover o dígito verificador e atualizar a malha)
for feature in geojson['features']:
    feature['properties']['codarea'] = feature['properties']['codarea']
geojson = rewind(geojson, rfc7946=False)


# Construir o mapa choroplético com uma barra de cores fixa e sem variação
fig2 = px.choropleth(variaveis_uf_ano,
                    geojson=geojson,
                    scope='south america',
                    color='gastos',
                    color_continuous_scale="agsunset_r",  # Escolha um mapa de cores adequado aqui
                    color_continuous_midpoint=100_000,  # Defina um valor médio para o mapa de cores
                    range_color=[15_000_000, 6_000_000_000],  # Defina manualmente o intervalo de cores
                    locations='cod_UF',
                    featureidkey='properties.codarea',
                    hover_name='uf',
                    animation_frame='ano')

fig2.update_geos(fitbounds="locations", visible=False)
fig2.update_traces(marker_line_width=0)

fig2.update_layout(height=800, width=1000, autosize=False, 
                  updatemenus=[{"buttons": [{"args": [None, {"frame": {"duration": 1000, "redraw": True}, "fromcurrent": True, "mode": "immediate", "transition": {"duration": 500, "easing": "linear"}}], "label": "Play", "method": "animate"}, 
                                          {"args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}], "label": "Stop", "method": "animate"}], 
                               "direction": "left", "pad": {"r": 10, "t": 87}, "showactive": False, "type": "buttons", "x": 0.1, "xanchor": "right", "y": 0, "yanchor": "top"}])

fig2.show()

In [83]:
def save_data(fig2):
    arquivo = 'C:/Users/Rosana/Documents/Rosana/Bootcamp_saude/fig2.pkl'
    with open(arquivo, 'wb') as pickle_file:
        pickle.dump(fig2, pickle_file)

In [59]:
base_completa = pd.read_csv('base_completa.csv', sep=';')
base_completa

,municipio,cod_ibge,cod_uf,uf,regiao,ano,gastos,mortalidade,aih_valor,dias,populacao,valor_emendas,gastos_percapita
0,ALTA FLORESTA D'OESTE,110001,11,RO,NORTE,2014,7.608883e+05,0.35,2296.0,5508.0,24066,3.181828e+07,31.616732
1,ARIQUEMES,110002,11,RO,NORTE,2014,3.463929e+06,2.57,5251.0,14364.0,99278,3.181828e+07,34.891200
2,CABIXI,110003,11,RO,NORTE,2014,8.940387e+04,NaN,282.0,646.0,5951,3.181828e+07,15.023336
3,CACOAL,110004,11,RO,NORTE,2014,7.392915e+06,3.59,10793.0,53025.0,82595,3.181828e+07,89.508017
4,CEREJEIRAS,110005,11,RO,NORTE,2014,4.911237e+05,0.37,1368.0,4154.0,16936,3.181828e+07,28.998798
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50125,VIANOPOLIS,522200,52,GO,CENTRO-OESTE,2022,2.859632e+05,1.44,975.0,3607.0,14046,1.156568e+08,20.359047
50126,VICENTINOPOLIS,522205,52,GO,CENTRO-OESTE,2022,1.212662e+05,2.33,258.0,545.0,8776,1.156568e+08,13.817932
50127,VILA BOA,522220,52,GO,CENTRO-OESTE,2022,NaN,NaN,NaN,NaN,3686,1.156568e+08,NaN
50128,VILA PROPICIO,522230,52,GO,CENTRO-OESTE,2022,NaN,NaN,NaN,NaN,5646,1.156568e+08,NaN


In [79]:

# Puxar a malha geográfica do Brasil a nível de município
geojson = requests.get(f'http://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=minima&intrarregiao=municipio').json()

# A malha geográfica do IBGE tem 7 dígitos (remover o dígito verificador e atualizar a malha)
for feature in geojson['features']:
    feature['properties']['codarea'] = feature['properties']['codarea'][:-1]
geojson = rewind(geojson, rfc7946=False)


# Construir o mapa choroplético com uma barra de cores fixa e sem variação
fig3 = px.choropleth(base_completa,
                    geojson=geojson,
                    scope='south america',
                    color='gastos',
                    color_continuous_scale="agsunset_r",  # Escolha um mapa de cores adequado aqui
                    color_continuous_midpoint=50_000,  # Defina um valor médio para o mapa de cores
                    range_color=[0, 1_560_000_000],  # Defina manualmente o intervalo de cores
                    locations='cod_ibge',
                    featureidkey='properties.codarea',
                    hover_name='municipio',
                   animation_frame='ano')

fig3.update_geos(fitbounds="locations", visible=False)
fig3.update_traces(marker_line_width=0)

fig3.update_layout(height=800, width=1000, autosize=False, 
                  updatemenus=[{"buttons": [{"args": [None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True, "mode": "immediate", "transition": {"duration": 300, "easing": "linear"}}], "label": "Play", "method": "animate"}, 
                                          {"args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}], "label": "Stop", "method": "animate"}], 
                               "direction": "left", "pad": {"r": 10, "t": 87}, "showactive": False, "type": "buttons", "x": 0.1, "xanchor": "right", "y": 0, "yanchor": "top"}])

fig3.show()


In [84]:
def save_data(fig3):
    arquivo = 'C:/Users/Rosana/Documents/Rosana/Bootcamp_saude/fig3.pkl'
    with open(arquivo, 'wb') as pickle_file:
        pickle.dump(fig3, pickle_file)

In [81]:

# Puxar a malha geográfica do Brasil a nível de município
geojson = requests.get(f'http://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=minima&intrarregiao=municipio').json()

# A malha geográfica do IBGE tem 7 dígitos (remover o dígito verificador e atualizar a malha)
for feature in geojson['features']:
    feature['properties']['codarea'] = feature['properties']['codarea'][:-1]
geojson = rewind(geojson, rfc7946=False)


# Construir o mapa choroplético com uma barra de cores fixa e sem variação
fig4 = px.choropleth(base_completa,
                    geojson=geojson,
                    scope='south america',
                    color='gastos_percapita',
                    color_continuous_scale="agsunset_r",  # Escolha um mapa de cores adequado aqui
                    color_continuous_midpoint=1_000,  # Defina um valor médio para o mapa de cores
                    range_color=[0, 3_700],  # Defina manualmente o intervalo de cores
                    locations='cod_ibge',
                    featureidkey='properties.codarea',
                    hover_name='municipio',
                   animation_frame='ano')

fig4.update_geos(fitbounds="locations", visible=False)
fig4.update_traces(marker_line_width=0)

fig4.update_layout(height=800, width=1000, autosize=False, 
                  updatemenus=[{"buttons": [{"args": [None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True, "mode": "immediate", "transition": {"duration": 300, "easing": "linear"}}], "label": "Play", "method": "animate"}, 
                                          {"args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}], "label": "Stop", "method": "animate"}], 
                               "direction": "left", "pad": {"r": 10, "t": 87}, "showactive": False, "type": "buttons", "x": 0.1, "xanchor": "right", "y": 0, "yanchor": "top"}])

fig4.show()


In [85]:
def save_data(fig4):
    arquivo = 'C:/Users/Rosana/Documents/Rosana/Bootcamp_saude/fig4.pkl'
    with open(arquivo, 'wb') as pickle_file:
        pickle.dump(fig4, pickle_file)